In [103]:
#Standard libraries
import pandas as pd
import numpy as np
import seaborn as sns

#Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder

#Metrics
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.pipeline import Pipeline
import pickle


Train = pd.read_csv("/Users/nikolajpettersson/Downloads/UNSW_NB15_training-set.csv")
test = pd.read_csv("/Users/nikolajpettersson/Downloads/UNSW_NB15_testing-set.csv")

df = pd.concat([Train,test], axis = 0)

/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3171553346.py:12: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfc = pd.read_csv(path)
/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3171553346.py:12: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfc = pd.read_csv(path)


In [90]:
#remove "-" entries from service
df = df[df['service'] != '-']

df1=df.copy() # To be used later

#drop irrelevant columns, hence id and the target column attack_category
drop_column = ["id", "attack_cat"]
df.drop(drop_column, axis=1, inplace=True)

# apply one-hot-encoding to categorical data (service,proto,state)
df_categorical = df.select_dtypes(exclude=[np.number])
print(df_categorical.columns)
for feature in df_categorical.columns:
    df[feature] = LabelEncoder().fit_transform(df[feature])

Index(['proto', 'service', 'state'], dtype='object')


In [92]:
from sklearn.preprocessing import StandardScaler

numerical_columns = df.select_dtypes(include=[np.number]).columns

scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# Working with the raw file

In [202]:
# Define the file names and paths
import os
file_names = ['UNSW-NB15_1.csv', 'UNSW-NB15_2.csv', 'UNSW-NB15_3.csv', 'UNSW-NB15_4.csv']
file_paths = [os.path.join('/Users/nikolajpettersson/Downloads',name) for name in file_names]


# Create an empty list to store the DataFrames
dfs = []

# Loop through the file paths and read in each CSV file as a DataFrame
for path in file_paths:
    dfc = pd.read_csv(path, header = None, names = feature_names, error_bad_lines = False)
    dfs.append(dfc)


# Read the CSV file into a DataFrame
names_df = pd.read_csv("/Users/nikolajpettersson/Downloads/NUSW-NB15_features_test.csv")

# Extract column names into a list
feature_names = names_df["Name"].tolist()

# Concatenate the DataFrames into a single DataFrame and add names
combined_df = pd.concat(dfs, ignore_index=True)

/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3768032848.py:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dfc = pd.read_csv(path, header = None, names = feature_names, error_bad_lines = False)
/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3768032848.py:12: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  dfc = pd.read_csv(path, header = None, names = feature_names, error_bad_lines = False)
/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3768032848.py:12: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  dfc = pd.read_csv(path, header = None, names = feature_names, error_bad_lines = False)
/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3768032848.py:12: DtypeWarning: Colu

In [203]:
combined_df.describe()

,dur,sbytes,dbytes,sttl,dttl,sloss,dloss,Sload,Dload,Spkts,...,ct_flw_http_mthd,is_ftp_login,ct_srv_src,ct_srv_dst,ct_dst_ltm,ct_src_ ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,Label
count,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,...,1.191902e+06,1.110168e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06,2.540047e+06
mean,6.587916e-01,4.339600e+03,3.642759e+04,6.278197e+01,3.076681e+01,5.163921e+00,1.632944e+01,3.695645e+07,2.450861e+06,3.328884e+01,...,2.345856e-01,3.969940e-02,9.206988e+00,8.988958e+00,6.439103e+00,6.900986e+00,4.642139e+00,3.592729e+00,6.845886e+00,1.264870e-01
std,1.392493e+01,5.640599e+04,1.610960e+05,7.462277e+01,4.285089e+01,2.251707e+01,5.659474e+01,1.186043e+08,4.224863e+06,7.628388e+01,...,7.940924e-01,1.996589e-01,1.083676e+01,1.082249e+01,8.162034e+00,8.205062e+00,8.477579e+00,6.174445e+00,1.125828e+01,3.323975e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,1.037000e-03,2.000000e+02,1.780000e+02,3.100000e+01,2.900000e+01,0.000000e+00,0.000000e+00,1.353963e+05,1.191594e+04,2.000000e+00,...,0.000000e+00,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
50%,1.586100e-02,1.470000e+03,1.820000e+03,3.100000e+01,2.900000e+01,3.000000e+00,4.000000e+00,5.893038e+05,5.893179e+05,1.200000e+01,...,0.000000e+00,0.000000e+00,5.000000e+00,5.000000e+00,3.000000e+00,4.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,0.000000e+00
75%,2.145545e-01,3.182000e+03,1.489400e+04,3.100000e+01,2.900000e+01,7.000000e+00,1.400000e+01,2.039923e+06,2.925974e+06,4.400000e+01,...,0.000000e+00,0.000000e+00,1.000000e+01,1.000000e+01,6.000000e+00,7.000000e+00,2.000000e+00,1.000000e+00,5.000000e+00,0.000000e+00
max,8.786638e+03,1.435577e+07,1.465753e+07,2.550000e+02,2.540000e+02,5.319000e+03,5.507000e+03,5.988000e+09,1.287619e+08,1.064600e+04,...,3.600000e+01,4.000000e+00,6.700000e+01,6.700000e+01,6.700000e+01,6.700000e+01,6.700000e+01,6.000000e+01,6.700000e+01,1.000000e+00


In [161]:
import pandas
import graphistry
import re

try:
    from urllib.parse import unquote # Python 3
except ImportError:
    from urllib import unquote       # Python 2

graphistry.register(api=3, protocol="https", server="hub.graphistry.com", username="testjupyter", password="!@#123qwe")

In [162]:
# Read the CSV file into a DataFrame
names_df = pd.read_csv("/Users/nikolajpettersson/Downloads/NUSW-NB15_features_test.csv")

# Extract column names into a list
feature_names = names_df["Name"].tolist()
feature_names

['srcip',
 'sport',
 'dstip',
 'dsport',
 'proto',
 'state',
 'dur',
 'sbytes',
 'dbytes',
 'sttl',
 'dttl',
 'sloss',
 'dloss',
 'service',
 'Sload',
 'Dload',
 'Spkts',
 'Dpkts',
 'swin',
 'dwin',
 'stcpb',
 'dtcpb',
 'smeansz',
 'dmeansz',
 'trans_depth',
 'res_bdy_len',
 'Sjit',
 'Djit',
 'Stime',
 'Ltime',
 'Sintpkt',
 'Dintpkt',
 'tcprtt',
 'synack',
 'ackdat',
 'is_sm_ips_ports',
 'ct_state_ttl',
 'ct_flw_http_mthd',
 'is_ftp_login',
 'ct_ftp_cmd',
 'ct_srv_src',
 'ct_srv_dst',
 'ct_dst_ltm',
 'ct_src_ ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'attack_cat',
 'Label']

In [106]:
df_raw = pd.read_csv("/Users/nikolajpettersson/Downloads/UNSW-NB15_2.csv", header= None,  names = feature_names)

/var/folders/ww/ffg0w_ss02z0ly4_2qtwpq2h0000gn/T/ipykernel_58041/3198927165.py:1: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv("/Users/nikolajpettersson/Downloads/UNSW-NB15_2.csv", header= None,  names = feature_names)


In [97]:
df_raw["attack_cat"].value_counts()

Generic             27883
Exploits            11103
 Fuzzers             4668
DoS                  4637
 Reconnaissance      3116
Analysis              608
Backdoor              370
 Shellcode            324
Worms                  40
Name: attack_cat, dtype: int64

In [128]:

grouped_df_raw = df_raw.groupby([df_raw.service, df_raw.srcip]).count()
grouped_df_raw.iloc[:len(grouped_df_raw)]

sport  dstip  dsport  proto  state   dur  sbytes  \
service srcip                                                            
-       10.40.170.2     702    702     702    702    702   702     702   
        10.40.182.1    1344   1344    1344   1344   1344  1344    1344   
        10.40.182.3     705    705     705    705    705   705     705   
        10.40.182.6     902    902     902    902    902   902     902   
        10.40.85.1     1335   1335    1335   1335   1335  1335    1335   
...                     ...    ...     ...    ...    ...   ...     ...   
ssh     59.166.0.9     1465   1465    1465   1465   1465  1465    1465   
ssl     175.45.176.0     13     13      13     13     13    13      13   
        175.45.176.1      3      3       3      3      3     3       3   
        175.45.176.2     13     13      13     13     13    13      13   
        175.45.176.3      3      3       3      3      3     3       3   

                      dbytes  sttl  dttl  ...  ct_ftp_cmd  ct_srv_src  \
service srcip                             ...                           
-       10.40.170.2      702   702   702  ...         702         702   
        10.40.182.1     1344  1344  1344  ...        1344        1344   
        10.40.182.3      705   705   705  ...         705         705   
        10.40.182.6      902   902   902  ...         902         902   
        10.40.85.1      1335  1335  1335  ...        1335        1335   
...                      ...   ...   ...  ...         ...         ...   
ssh     59.166.0.9      1465  1465  1465  ...        1465        1465   
ssl     175.45.176.0      13    13    13  ...          13          13   
        175.45.176.1       3     3     3  ...           3           3   
        175.45.176.2      13    13    13  ...          13          13   
        175.45.176.3       3     3     3  ...           3           3   

                      ct_srv_dst  ct_dst_ltm  ct_src_ ltm  ct_src_dport_ltm  \
service srcip                                                                 
-       10.40.170.2          702         702          702               702   
        10.40.182.1         1344        1344         1344              1344   
        10.40.182.3          705         705          705               705   
        10.40.182.6          902         902          902               902   
        10.40.85.1          1335        1335         1335              1335   
...                          ...         ...          ...               ...   
ssh     59.166.0.9          1465        1465         1465              1465   
ssl     175.45.176.0          13          13           13                13   
        175.45.176.1           3           3            3                 3   
        175.45.176.2          13          13           13                13   
        175.45.176.3           3           3            3                 3   

                      ct_dst_sport_ltm  ct_dst_src_ltm  attack_cat  Label  
service srcip                                                              
-       10.40.170.2                702             702           0    702  
        10.40.182.1               1344            1344           0   1344  
        10.40.182.3                705             705           0    705  
        10.40.182.6                902             902           0    902  
        10.40.85.1                1335            1335           0   1335  
...                                ...             ...         ...    ...  
ssh     59.166.0.9                1465            1465           0   1465  
ssl     175.45.176.0                13              13          13     13  
        175.45.176.1                 3               3           3      3  
        175.45.176.2                13              13          13     13  
        175.45.176.3                 3               3           3      3  

[153 rows x 47 columns]

In [159]:
summary.columns

Index(['srcip', 'service', 'dloss', 'proto', 'attack_cat', 'count',
       'most_frequent_attack_cat'],
      dtype='object')

In [154]:
#Bundle edges into a Pandas group when they share the same attributes like 'host' and 'path'
grouped_logs = grouped_df_raw.groupby(['srcip', 'service', 'dloss', 'proto', 'attack_cat'])

# Make dataframes count, min_time, max_time, and referer that are indexed by the groupby keys.
count = grouped_logs.size().to_frame('count')
#min_time = grouped_logs['time'].agg('min').to_frame('time (min)')
#max_time = grouped_logs['time'].agg('max').to_frame('time (max)')

def mostFrequent(x):
    s = x.value_counts()
    return None#return s.index[0] if len(s.index > 0) else None
attack_cat_1 = grouped_logs['attack_cat'].agg(mostFrequent)

# Join into one table based on the same groupby keys
# We remove the indexes (via reset_index) since we do not need them anymore.
summary = count.join([attack_cat_1]).reset_index()

def path2pathGraph(grouped_logs):
    host2path = grouped_logs[['srcip', 'service']].copy()
    host2path['srcip'] = host2path['srcip'].map(lambda p: p.split('?')[0])
    sessions = pandas.merge(host2path, host2path, on='srcip').drop_duplicates()

    host2color = {host: 265000 + index for index, host in enumerate(sessions.host.unique())}
    sessions['ecolor'] = sessions['srcip'].map(lambda x: host2color[x])
    return sessions

sessionEdges = path2pathGraph(count)


ValueError: cannot insert attack_cat, already exists

In [175]:
#df_raw_graph = df_raw[['service', 'attack_cat',]].drop_duplicates().drop_na()

df_raw_graph= df_raw.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)


In [178]:
#select items that will be used graphing. 
#df_raw_graph = df_raw[['service', 'attack_cat',]].drop_duplicates().drop_na()

df_raw_graph= df_raw.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='attack_cat', destination='service').edges(df_raw_graph)
g.plot()

       service attack_cat
count       64         54
unique      13          9
top          -   Exploits
freq        10         13


In [179]:

df_raw_graph= df_raw.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='service', destination='attack_cat').edges(df_raw_graph)
g.plot()

In [200]:
import pandas as pd

# Select the specific columns of interest
selected_columns = combined_df.loc[:,["service", "attack_cat"]]

# Create a column indicating duplicate occurrences
combined_df['duplicate_count'] = selected_columns.duplicated(keep=False)

# Group by the columns of interest and calculate the counts for each group
combined_df['duplicate_count'] = combined_df.groupby(["service", "attack_cat"])['duplicate_count'].transform('sum')

# Print the resulting DataFrame
print(combined_df)


                srcip  sport           dstip dsport proto state       dur  \
0          59.166.0.0   1390   149.171.126.6     53   udp   CON  0.001055   
1          59.166.0.0  33661   149.171.126.9   1024   udp   CON  0.036133   
2          59.166.0.6   1464   149.171.126.7     53   udp   CON  0.001119   
3          59.166.0.5   3593   149.171.126.5     53   udp   CON  0.001209   
4          59.166.0.3  49664   149.171.126.0     53   udp   CON  0.001169   
...               ...    ...             ...    ...   ...   ...       ...   
2540042    59.166.0.5  33094   149.171.126.7  43433   tcp   FIN  0.087306   
2540043    59.166.0.7  20848   149.171.126.4     21   tcp   CON  0.365058   
2540044    59.166.0.3  21511   149.171.126.9     21   tcp   CON  6.335154   
2540045    59.166.0.9  35433   149.171.126.0     80   tcp   CON  2.200934   
2540046  175.45.176.0  17293  149.171.126.17    110   tcp   CON  0.942984   

         sbytes  dbytes  sttl  ...  ct_srv_src  ct_srv_dst  ct_dst_ltm  \
0

In [230]:

df_graph = combined_df.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='service', destination='attack_cat').edges(df_graph)
g.plot()

#exploits and generic datasets are use a lot of differnt known sevices, while things as shellcode only works with fewer things


In [229]:

df_graph = combined_df.loc[:, ["service", "attack_cat"]].reset_index().drop(["index"],axis = 1).drop_duplicates()

#df_graph = combined_df.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='service', destination='attack_cat').edges(df_graph)
g.plot()

#note that the IRC service is not used by generic traffic. 

In [212]:
combined_df.columns

Index(['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes',
       'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload',
       'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz',
       'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime',
       'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat',
       'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login',
       'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat',
       'Label'],
      dtype='object')

In [226]:
df_graph = combined_df.loc[:, ["state", "attack_cat"]].reset_index().drop(["index"],axis = 1).drop_duplicates()

#df_graph = combined_df.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='state', destination='attack_cat').edges(df_graph)
g.plot()

In [232]:
df_graph = combined_df.loc[:, ["dstip", "attack_cat"]].reset_index().drop(["index"],axis = 1).drop_duplicates()

#df_graph = combined_df.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='attack_cat', destination='dstip').edges(df_graph)
g.plot()

#Utterly useless, the attacks come from all the IP adresses, which is to be expected as this is a controlled experiment.
# however in reality unknown IP adresses, specifically compared to Generic traffic can serve as a warning. 

In [235]:
df_graph = combined_df.loc[:, ["proto", "service"]].reset_index().drop(["index"],axis = 1).drop_duplicates()

#df_graph = combined_df.loc[:, ["service","attack_cat"]].drop_duplicates().reset_index().drop(["index"],axis=1)

from graphistry import *
g=graphistry.bind(source='service', destination='proto').edges(df_graph)
g.plot()

Failed to post arrow to api/v2/upload/datasets/998da795c92d4abab4e3407ea9f04a5e/edges/arrow (https://hub.graphistry.com/api/v2/upload/datasets/998da795c92d4abab4e3407ea9f04a5e/edges/arrow)
Traceback (most recent call last):
  File "/Users/nikolajpettersson/opt/anaconda3/lib/python3.9/site-packages/graphistry/arrow_uploader.py", line 655, in post_arrow
    resp = self.post_arrow_generic(sub_path, tok, arr, opts)
  File "/Users/nikolajpettersson/opt/anaconda3/lib/python3.9/site-packages/graphistry/arrow_uploader.py", line 684, in post_arrow_generic
    resp.raise_for_status()
  File "/Users/nikolajpettersson/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 422 Client Error: Unprocessable Entity for url: https://hub.graphistry.com/api/v2/upload/datasets/998da795c92d4abab4e3407ea9f04a5e/edges/arrow422 Client Error: Unprocessable Entity for url: https://hub.graphist

HTTPError: 422 Client Error: Unprocessable Entity for url: https://hub.graphistry.com/api/v2/upload/datasets/998da795c92d4abab4e3407ea9f04a5e/edges/arrow